<a href="https://colab.research.google.com/github/koroltony/pyrtltest/blob/main/PyRTLFIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Pyrtl in Colab

In [1]:
!pip install pyrtl


## Build Circuit in Pyrtl

In [2]:
import pyrtl

def fir(x, bs):
    rwidth = len(x)  # bitwidth of the registers
    ntaps = len(bs) # number of coefficients

    zs = [x] + [pyrtl.Register(rwidth) for _ in range(ntaps-1)]
    for i in range(1,ntaps):
        zs[i].next <<= zs[i-1]

    # produce the final sum of products
    return sum(z*b for z,b in zip(zs, bs))

x = pyrtl.Input(8, 'x')
y = pyrtl.Output(8, 'y')
y <<= fir(x, [0, 1])

sim = pyrtl.Simulation()
sim.step_multiple({'x':[0, 9, 18, 8, 17, 7, 16, 6, 15, 5]})
sim.tracer.render_trace()

#create verilog file
with open('hardware.v', 'w') as file:
    pyrtl.output_to_verilog(file, add_reset=True, block=None)

  ▏0                        ▏5                       

x 0x0  ╳0x9 ╳0x12╳0x8 ╳0x11 ╳0x7 ╳0x10╳0x6 ╳0xf ╳0x5 

y 0x0       ╳0x9 ╳0x12╳0x8  ╳0x11╳0x7 ╳0x10╳0x6 ╳0xf 



## Waveform simulation

In [4]:
#@title Waveform Viewer
import json
from IPython.display import IFrame

def generate_waveform_data(sim_trace, hex_output=False):
    waveform_signals = []

    for signal_name, signal_values in sim_trace.trace.items():
        waveform_signal = {
            "name": signal_name,
            "wave": "2" * len(signal_values),
            "data": [hex(value) if hex_output else str(value) for value in signal_values]
        }
        waveform_signals.append(waveform_signal)
    #visual clock signal on WaveDrom
    waveform_clock = {
      'name': 'clock',
      'wave': 'p' * len(signal_values)
    }
    waveform_signals.append(waveform_clock)

    waveform_data = {
        "signal": waveform_signals,
        "foot": {
            "tick": 0
        }
    }
    return waveform_data

# Generate waveform data (Choose hex true or false)
waveform_data_hex = generate_waveform_data(sim.tracer, hex_output=True)
waveform_data_decimal = generate_waveform_data(sim.tracer, hex_output=False)

# Save the waveform data to a JSON file (to open in WaveDrom website)
with open('waveform_hex.json', 'w') as file:
    json.dump(waveform_data_hex, file, indent=2)

with open('waveform_dec.json', 'w') as file:
    json.dump(waveform_data_decimal, file, indent=2)

print("Waveforms saved to 'waveform_---.json'")
print("-> Copy paste the contents of either of the JSON files into the text box below")
print("   (click on the folder icon on the left of the screen to see the files)")
print("-> Make sure to clear the original output as it may be from previous circuits")
print("-> Note, the waveform is offset due to real-life delay in hardware")

# Display the WaveDrom website in Colab
IFrame(src='https://wavedrom.com/editor.html', width='100%', height='500px')


Waveforms saved to 'waveform_---.json'
-> Copy paste the contents of either of the JSON files into the text box below
   (click on the folder icon on the left of the screen to see the files)
-> Make sure to clear the original output as it may be from previous circuits
-> Note, the waveform is offset due to real-life delay in hardware
